# Data-Driven Drone Altitude Control System

## Integrated Pipeline: Data Preparation → Visualization → Model Training → MPC Control → Trajectory Analysis

This notebook implements a complete control system combining:
- **Data Processing**: Loading and normalizing control inputs/outputs
- **Exploratory Analysis**: Statistical plots and frequency analysis
- **Sequence Modeling**: LSTM, GRU, or Transformer architectures
- **Mathematical Framework**: State-space modeling and system identification
- **Model Predictive Control (MPC)**: Trajectory optimization for altitude commands
- **Interactive Control**: Design target heights and simulate system response

## Section 1: Setup and Data Loading

In [ ]:
# Core imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Deep learning
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Control theory
from scipy.linalg import solve_discrete_are
from scipy.signal import butter, filtfilt

# Visualization
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set paths
DATA_DIR = Path('.')
input_file = DATA_DIR / 'bdd_in_mat_05.csv'
output_file = DATA_DIR / 'bdd_out_mat_05.csv'

## Section 2: Data Preparation and Exploration

In [ ]:
# Load data
print("Loading data...")
u_raw = np.genfromtxt(input_file, delimiter=',')
y_raw = np.genfromtxt(output_file, delimiter=',')

# Reshape to column vectors if needed
if u_raw.ndim == 1:
    u_raw = u_raw.reshape(-1, 1)
if y_raw.ndim == 1:
    y_raw = y_raw.reshape(-1, 1)

print(f"Input shape: {u_raw.shape}")
print(f"Output shape: {y_raw.shape}")
print(f"Data duration: {u_raw.shape[0] * 0.05:.1f} seconds (Ts=50ms)")

In [ ]:
# Statistical analysis
print("\n=== INPUT STATISTICS ===")
print(f"Mean: {u_raw.mean():.6f}")
print(f"Std:  {u_raw.std():.6f}")
print(f"Min:  {u_raw.min():.6f}")
print(f"Max:  {u_raw.max():.6f}")

print("\n=== OUTPUT STATISTICS ===")
print(f"Mean: {y_raw.mean():.6f}")
print(f"Std:  {y_raw.std():.6f}")
print(f"Min:  {y_raw.min():.6f}")
print(f"Max:  {y_raw.max():.6f}")

In [ ]:
# Visualization: Time series
fig, axes = plt.subplots(3, 1, figsize=(15, 8))

# Commands (inputs)
axes[0].plot(u_raw, 'b-', linewidth=0.7, alpha=0.8)
axes[0].set_ylabel('Command u(t) [normalized]', fontsize=11)
axes[0].set_title('Input Commands Over Time', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Outputs
axes[1].plot(y_raw, 'g-', linewidth=0.7, alpha=0.8)
axes[1].set_ylabel('Output y(t) [acceleration m/s²]', fontsize=11)
axes[1].set_title('System Output (Acceleration)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Phase plane: u vs y
axes[2].scatter(u_raw[:1000], y_raw[:1000], alpha=0.5, s=10, c=np.arange(1000), cmap='viridis')
axes[2].set_xlabel('Command u(t)', fontsize=11)
axes[2].set_ylabel('Output y(t)', fontsize=11)
axes[2].set_title('Phase Plane (Input-Output Relationship)', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('01_data_exploration.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Data exploration plot saved")

In [ ]:
# Normalization for neural networks
scaler_u = StandardScaler()
scaler_y = StandardScaler()

u_scaled = scaler_u.fit_transform(u_raw)
y_scaled = scaler_y.fit_transform(y_raw)

print("Scalers fitted:")
print(f"  Input: mean={scaler_u.mean_[0]:.6f}, scale={scaler_u.scale_[0]:.6f}")
print(f"  Output: mean={scaler_y.mean_[0]:.6f}, scale={scaler_y.scale_[0]:.6f}")

## Section 3: Mathematical Framework - System Identification

In [ ]:
# ============================================================
# MATHEMATICAL BACKGROUND: State-Space Drone Model
# ============================================================

math_text = r"""
DISCRETE-TIME STATE-SPACE MODEL:
  x_{k+1} = A x_k + B u_k
  y_k = C x_k + D u_k

DRONE ALTITUDE DYNAMICS:
  State: x_k = [z_k, v_k, a_k]ᵀ  (position, velocity, acceleration)
  Input: u_k ∈ [-1, 1]  (normalized thrust command)
  Output: y_k = a_k  (measured acceleration)

CONTINUOUS FORM (for reference):
  z̈ = a  (double integration)
  τ·ȧ + a = u  (first-order thrust dynamics)

AUGMENTED LQI SYSTEM (with integrator state):
  χ_k = [x_k; η_k]  where η_k = ∫(z_ref - z_k)dt (tracking error integral)
  
COST FUNCTION:
  J = Σ(χₖᵀ Q χₖ + uₖᵀ R uₖ)
  Q: penalize position error (large) > velocity (medium) > acceleration (small)
  R: penalize control effort

LQI CONTROL LAW:
  u_k = -K χ̂_k  where K is computed via discrete Riccati equation

KALMAN OBSERVER (discrete):
  Correction step: χ̂ₖ = χ̂ₖ⁻ + L(yₖ - ŷₖ)
  L = solution to dual Riccati with process/measurement noise covariances
"""

print(math_text)

In [ ]:
# ============================================================
# SYSTEM IDENTIFICATION: Estimate A, B, C, D matrices
# ============================================================

def identify_system(u_data, y_data, Ts=0.05):
    """
    Estimate discrete-time LTI system from input-output data.
    Simple assumption: first-order acceleration model.
    """
    # Integrate acceleration to get velocity and position
    a = y_data.flatten()
    v = np.cumsum(a) * Ts
    z = np.cumsum(v) * Ts
    
    # Construct regressor matrix: [a_{k-1}, v_{k-1}, z_{k-1}, u_{k-1}]
    u = u_data.flatten()
    n_samples = len(a) - 1
    
    # Regressor: past states and input
    Phi = np.column_stack([a[:-1], v[:-1], z[:-1], u[:-1], np.ones(n_samples)])
    
    # Target: current acceleration
    a_target = a[1:]
    
    # Least squares: minimize ||a_target - Phi @ theta||
    theta = np.linalg.lstsq(Phi, a_target, rcond=None)[0]
    
    print("\nSystem Identification Results:")
    print(f"  a_k = {theta[0]:.4f}*a_(k-1) + {theta[1]:.6f}*v_(k-1) + {theta[2]:.6f}*z_(k-1) + {theta[3]:.4f}*u_(k-1) + {theta[4]:.6f}")
    
    # Construct state-space
    A = np.array([
        [1.0, Ts, 0.0],      # z_{k+1} = z_k + v_k * Ts
        [0.0, 1.0, Ts],      # v_{k+1} = v_k + a_k * Ts
        [0.0, 0.0, theta[0]] # a_{k+1} = theta[0] * a_k
    ])
    
    B = np.array([[0.0], [0.0], [theta[3]]])  # a_k influenced by u_k
    C = np.array([[0.0, 0.0, 1.0]])           # Output is acceleration
    D = np.array([[0.0]])
    
    return A, B, C, D, theta

A_id, B_id, C_id, D_id, theta_id = identify_system(u_raw, y_raw)

In [ ]:
# Visualize system identification
print("\nIdentified Discrete-Time Matrices (A, B, C, D):")
print(f"\nA =\n{A_id}")
print(f"\nB =\n{B_id}")
print(f"\nC =\n{C_id}")
print(f"\nD =\n{D_id}")

## Section 4: Sequence Learning - Model Architecture Selection

In [ ]:
# ============================================================
# NEURAL NETWORK MODELS
# ============================================================

class LSTMModel(nn.Module):
    """LSTM-based sequence-to-sequence model for system identification."""
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        y = self.fc(lstm_out)
        return y

class GRUModel(nn.Module):
    """GRU-based sequence model (simpler than LSTM, often faster)."""
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, output_size=1):
        super().__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
    
    def forward(self, x):
        gru_out, _ = self.gru(x)
        y = self.fc(gru_out)
        return y

class TransformerModel(nn.Module):
    """Transformer-based encoder for sequence modeling."""
    def __init__(self, input_size=1, d_model=64, nhead=4, num_layers=2, output_size=1):
        super().__init__()
        self.embedding = nn.Linear(input_size, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=256, batch_first=True, dropout=0.2)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, output_size)
        )
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        y = self.fc(x)
        return y

print("✓ Model classes defined: LSTM, GRU, Transformer")

In [ ]:
# ============================================================
# PREPARE SEQUENCES FOR TRAINING
# ============================================================

def create_sequences(u_data, y_data, seq_length=20):
    """
    Create sequences: X[k:k+seq_length] -> y[k+seq_length]
    Maps past seq_length steps of input/output to next output.
    """
    X, Y = [], []
    for i in range(len(u_data) - seq_length):
        # Input sequence: concatenate u and y history
        seq_u = u_data[i:i+seq_length]
        seq_y = y_data[i:i+seq_length]
        combined = np.hstack([seq_u, seq_y])  # Shape: (seq_length, 2)
        X.append(combined)
        # Target: next output
        Y.append(y_data[i+seq_length])
    return np.array(X), np.array(Y)

SEQ_LENGTH = 20
X, Y = create_sequences(u_scaled, y_scaled, SEQ_LENGTH)
print(f"Sequences created: X.shape={X.shape}, Y.shape={Y.shape}")
print(f"  Number of samples: {X.shape[0]}")
print(f"  Sequence length: {X.shape[1]}")

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# Convert to torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).unsqueeze(-1)

print(f"Training set: {X_train_t.shape}")
print(f"Test set: {X_test_t.shape}")

In [ ]:
# ============================================================
# SELECT MODEL ARCHITECTURE
# ============================================================

print("\n" + "="*60)
print("SELECT YOUR NEURAL NETWORK MODEL")
print("="*60)
print("""
Options:
  1. LSTM (Long Short-Term Memory)
     - Pros: Excellent for long sequences, handles vanishing gradients
     - Cons: Slower training, more parameters
     
  2. GRU (Gated Recurrent Unit)
     - Pros: Similar to LSTM but simpler, faster training
     - Cons: Slightly less capable than LSTM for very long sequences
     
  3. Transformer
     - Pros: Parallel processing, modern architecture, excellent for patterns
     - Cons: Requires more data, higher memory
""")

# Default to LSTM, change this value to select different model
MODEL_TYPE = "LSTM"  # Options: "LSTM", "GRU", "Transformer"

print(f"\n>>> Selected: {MODEL_TYPE}")
print("   (Change MODEL_TYPE variable to switch architectures)")

In [ ]:
# ============================================================
# TRAIN THE SELECTED MODEL
# ============================================================

def train_model(model, X_train, y_train, X_test, y_test, epochs=50, batch_size=32, lr=1e-3):
    """Training loop with validation."""
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=False)
    
    train_losses = []
    test_losses = []
    
    model.to(device)
    
    for epoch in range(epochs):
        # Train
        model.train()
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        train_loss = epoch_loss / len(train_loader)
        
        # Evaluate
        model.eval()
        with torch.no_grad():
            y_pred_test = model(X_test.to(device))
            test_loss = criterion(y_pred_test, y_test.to(device)).item()
        
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        scheduler.step(test_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1:3d}/{epochs} | Train Loss: {train_loss:.6f} | Test Loss: {test_loss:.6f}")
    
    return model, train_losses, test_losses

# Initialize model based on selection
if MODEL_TYPE == "LSTM":
    model = LSTMModel(input_size=2, hidden_size=64, num_layers=2, output_size=1)
elif MODEL_TYPE == "GRU":
    model = GRUModel(input_size=2, hidden_size=64, num_layers=2, output_size=1)
else:  # Transformer
    model = TransformerModel(input_size=2, d_model=64, nhead=4, num_layers=2, output_size=1)

print(f"\nTraining {MODEL_TYPE} model...")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

model, train_losses, test_losses = train_model(
    model, X_train_t, y_train_t, X_test_t, y_test_t,
    epochs=50, batch_size=32, lr=1e-3
)

print("\n✓ Training complete!")

In [ ]:
# Plot training history
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(train_losses, label='Train Loss', linewidth=2, marker='o', markersize=4, alpha=0.7)
ax.plot(test_losses, label='Test Loss', linewidth=2, marker='s', markersize=4, alpha=0.7)
ax.set_xlabel('Epoch', fontsize=11)
ax.set_ylabel('MSE Loss', fontsize=11)
ax.set_title(f'{MODEL_TYPE} Model - Training History', fontsize=12, fontweight='bold')
ax.set_yscale('log')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('02_training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Training history plot saved")

## Section 5: Model Predictive Control (MPC)

In [ ]:
# ============================================================
# MPC CONTROLLER FOR DRONE ALTITUDE
# ============================================================

class MPCController:
    """
    Model Predictive Controller for altitude reference tracking.
    
    Formulation:
      min_{u_k,...,u_{k+Hp-1}} Σ ||z_k - z_ref||_Q² + ||u_k||_R²
      subject to: z_{k+1} = A*z_k + B*u_k
                  u_min <= u_k <= u_max
    
    Uses LQI with augmented state including integrator of tracking error.
    """
    
    def __init__(self, A, B, C, D, Ts=0.05, Hp=10, z_ref_init=5.0):
        """
        Args:
            A, B, C, D: System matrices (discrete-time)
            Ts: Sampling time
            Hp: Prediction horizon
            z_ref_init: Initial altitude reference
        """
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        self.Ts = Ts
        self.Hp = Hp
        self.z_ref = z_ref_init
        
        # State: [z, v, a]
        self.state = np.array([0.0, 0.0, 0.0])
        self.z_history = []
        self.u_history = []
        
        # LQI gains (computed once)
        self._compute_gains()
    
    def _compute_gains(self):
        """Compute LQI controller and Kalman observer gains."""
        n = self.A.shape[0]  # 3
        
        # Augmented system: add integrator state η
        # χ = [z, v, a, η]
        A_aug = np.eye(n + 1)
        A_aug[:n, :n] = self.A
        A_aug[-1, 0] = 1.0  # η_{k+1} = η_k + z_k (tracking error integral)
        
        B_aug = np.vstack([self.B, np.zeros((1, 1))])
        
        # LQR cost
        Q = np.diag([100, 10, 1, 500])  # High penalty on position and integrator
        R = np.array([[1.0]])
        
        # Solve discrete ARE
        P = solve_discrete_are(A_aug, B_aug, Q, R)
        self.K = np.linalg.inv(R + B_aug.T @ P @ B_aug) @ B_aug.T @ P @ A_aug
    
    def step(self, z_measured, v_measured=None, a_measured=None):
        """
        Execute one control step.
        
        Args:
            z_measured: Measured position
            v_measured: Measured velocity (estimated if None)
            a_measured: Measured acceleration (estimated if None)
        
        Returns:
            u: Control command
            state_dict: Dictionary with current state estimates
        """
        # Update position from measurement
        self.state[0] = z_measured
        
        # Estimate velocity/acceleration if not provided
        if v_measured is not None:
            self.state[1] = v_measured
        if a_measured is not None:
            self.state[2] = a_measured
        
        # Augmented state with integrator
        error = self.z_ref - self.state[0]
        if not hasattr(self, 'integrator'):
            self.integrator = 0.0
        self.integrator += error * self.Ts
        
        chi = np.concatenate([self.state, [self.integrator]])
        
        # LQI control
        u = -self.K @ chi.reshape(-1, 1)
        u = float(np.clip(u, -1.0, 1.0))
        
        # Propagate state
        self.state = (self.A @ self.state.reshape(-1, 1) + self.B * u).flatten()
        
        # Log
        self.z_history.append(self.state[0])
        self.u_history.append(u)
        
        return u, {
            'z': self.state[0],
            'v': self.state[1],
            'a': self.state[2],
            'error': error,
            'integrator': self.integrator
        }
    
    def set_reference(self, z_ref):
        """Update altitude reference."""
        self.z_ref = z_ref
    
    def reset(self):
        """Reset controller state."""
        self.state = np.array([0.0, 0.0, 0.0])
        self.integrator = 0.0
        self.z_history = []
        self.u_history = []

# Create controller
mpc = MPCController(A_id, B_id, C_id, D_id, Ts=0.05, Hp=10, z_ref_init=5.0)
print("✓ MPC Controller initialized")
print(f"  LQI gain K shape: {mpc.K.shape}")

## Section 6: Interactive Control - Design Your Trajectory

In [ ]:
# ============================================================
# INTERACTIVE: DEFINE ALTITUDE REFERENCE TRAJECTORY
# ============================================================

print("\n" + "="*60)
print("DESIGN YOUR ALTITUDE REFERENCE TRAJECTORY")
print("="*60)

# Simple trajectory: step commands at different times
T_total = 20.0  # seconds
Ts = 0.05
num_steps = int(T_total / Ts)

# Define reference trajectory
z_ref_trajectory = np.zeros(num_steps)

# Example: Multi-step reference
# Segment 1: 0-5s at z=0
z_ref_trajectory[0:int(5/Ts)] = 0.0
# Segment 2: 5-10s at z=5m
z_ref_trajectory[int(5/Ts):int(10/Ts)] = 5.0
# Segment 3: 10-15s at z=8m
z_ref_trajectory[int(10/Ts):int(15/Ts)] = 8.0
# Segment 4: 15-20s at z=3m
z_ref_trajectory[int(15/Ts):] = 3.0

print(f"\nTrajectory definition:")
print(f"  0-5s:  z_ref = 0m   (hover at ground)")
print(f"  5-10s: z_ref = 5m   (climb)")
print(f"  10-15s: z_ref = 8m  (climb more)")
print(f"  15-20s: z_ref = 3m  (descend)")

print(f"\nTotal simulation time: {T_total} seconds")
print(f"Number of steps: {num_steps}")

In [ ]:
# ============================================================
# RUN MPC SIMULATION
# ============================================================

mpc.reset()
mpc.set_reference(z_ref_trajectory[0])

# Storage
z_trajectory = []
u_trajectory = []
error_trajectory = []
integrator_trajectory = []

print("\nSimulating MPC control...")
for k in range(num_steps):
    # Update reference
    mpc.set_reference(z_ref_trajectory[k])
    
    # Current position (with small noise)
    z_current = mpc.state[0]
    
    # Control step
    u, state_info = mpc.step(z_current)
    
    # Log
    z_trajectory.append(state_info['z'])
    u_trajectory.append(u)
    error_trajectory.append(state_info['error'])
    integrator_trajectory.append(state_info['integrator'])

z_trajectory = np.array(z_trajectory)
u_trajectory = np.array(u_trajectory)
error_trajectory = np.array(error_trajectory)
integrator_trajectory = np.array(integrator_trajectory)
time_axis = np.arange(num_steps) * Ts

print(f"✓ Simulation complete!")
print(f"  Final position: {z_trajectory[-1]:.3f} m")
print(f"  Final error: {error_trajectory[-1]:.3f} m")

In [ ]:
# ============================================================
# VISUALIZATION: MPC CONTROL RESULTS
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Position tracking
axes[0, 0].plot(time_axis, z_ref_trajectory, 'r--', linewidth=2, label='Reference z_ref(t)', alpha=0.8)
axes[0, 0].plot(time_axis, z_trajectory, 'b-', linewidth=2, label='Actual z(t)', alpha=0.8)
axes[0, 0].fill_between(time_axis, z_ref_trajectory - 0.5, z_ref_trajectory + 0.5, alpha=0.1, color='red')
axes[0, 0].set_ylabel('Altitude (m)', fontsize=11)
axes[0, 0].set_title('Altitude Tracking Performance', fontsize=12, fontweight='bold')
axes[0, 0].legend(fontsize=10, loc='best')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Control input
axes[0, 1].fill_between(time_axis, -1, 1, alpha=0.1, color='gray', label='Saturation limits')
axes[0, 1].plot(time_axis, u_trajectory, 'g-', linewidth=2, label='Control u(t)')
axes[0, 1].axhline(1.0, color='r', linestyle='--', alpha=0.5)
axes[0, 1].axhline(-1.0, color='r', linestyle='--', alpha=0.5)
axes[0, 1].set_ylabel('Control Input (normalized)', fontsize=11)
axes[0, 1].set_title('MPC Control Commands', fontsize=12, fontweight='bold')
axes[0, 1].legend(fontsize=10, loc='best')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Tracking error
axes[1, 0].plot(time_axis, error_trajectory, 'purple', linewidth=2, label='Tracking error e(t) = z_ref - z')
axes[1, 0].axhline(0, color='k', linestyle='-', alpha=0.3)
axes[1, 0].fill_between(time_axis, 0, error_trajectory, alpha=0.3, color='purple')
axes[1, 0].set_ylabel('Error (m)', fontsize=11)
axes[1, 0].set_title('Altitude Tracking Error', fontsize=12, fontweight='bold')
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Phase plane - error vs integrator
axes[1, 1].scatter(error_trajectory[::10], integrator_trajectory[::10], 
                   c=time_axis[::10], cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[1, 1].set_xlabel('Tracking Error (m)', fontsize=11)
axes[1, 1].set_ylabel('Integral of Error', fontsize=11)
axes[1, 1].set_title('Phase Plane: Error Dynamics', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
cbar = plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1])
cbar.set_label('Time (s)', fontsize=10)

plt.tight_layout()
plt.savefig('03_mpc_control_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ MPC results plot saved")

## Section 7: Performance Analysis and Metrics

In [ ]:
# ============================================================
# CONTROL PERFORMANCE METRICS
# ============================================================

# Compute metrics
IAE = np.sum(np.abs(error_trajectory)) * Ts  # Integral of absolute error
ISE = np.sum(error_trajectory**2) * Ts        # Integral of squared error
max_error = np.max(np.abs(error_trajectory))
steady_state_error = np.abs(error_trajectory[-1])
control_effort = np.sum(np.abs(u_trajectory))

# Settling time (when error < 5% of max)
threshold = 0.05 * np.max(np.abs(error_trajectory))
settled_idx = np.where(np.abs(error_trajectory) < threshold)[0]
settling_time = time_axis[settled_idx[0]] if len(settled_idx) > 0 else T_total

print("\n" + "="*60)
print("CONTROL PERFORMANCE METRICS")
print("="*60)
print(f"\nTracking Accuracy:")
print(f"  Integral Absolute Error (IAE):   {IAE:.4f} m·s")
print(f"  Integral Squared Error (ISE):    {ISE:.4f} m²·s")
print(f"  Maximum Tracking Error:          {max_error:.4f} m")
print(f"  Steady-State Error:              {steady_state_error:.4f} m")
print(f"  Settling Time (5%):              {settling_time:.2f} s")

print(f"\nControl Effort:")
print(f"  Total Control Effort:            {control_effort:.2f}")
print(f"  Average Command Magnitude:       {np.mean(np.abs(u_trajectory)):.4f}")
print(f"  Max Command Magnitude:           {np.max(np.abs(u_trajectory)):.4f}")

print(f"\nSystem Dynamics:")
print(f"  Mean Altitude Achieved:          {np.mean(z_trajectory):.4f} m")
print(f"  Altitude Standard Deviation:     {np.std(z_trajectory):.4f} m")

## Section 8: Comparison and Advanced Features

In [ ]:
# ============================================================
# NEURAL NETWORK MODEL PREDICTION (Optional)
# ============================================================

# Use the trained neural network to predict system behavior
model.eval()
with torch.no_grad():
    # Prepare input sequence from current trajectory
    if len(z_trajectory) >= SEQ_LENGTH:
        # Create input from last SEQ_LENGTH samples
        u_seq = u_trajectory[-SEQ_LENGTH:].reshape(-1, 1)
        y_seq = error_trajectory[-SEQ_LENGTH:].reshape(-1, 1)  # Or use derivative
        
        X_pred = np.hstack([u_seq, y_seq])
        X_pred_t = torch.tensor(X_pred, dtype=torch.float32).unsqueeze(0)
        
        y_pred_t = model(X_pred_t.to(device))
        y_pred_scaled = y_pred_t.cpu().numpy().flatten()
        
        # Inverse transform
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
        
        print(f"\nNeural Network Prediction:")
        print(f"  Next acceleration prediction: {y_pred[0, 0]:.6f} m/s²")
    else:
        print(f"\n(Need at least {SEQ_LENGTH} samples to make predictions)")

In [ ]:
# ============================================================
# SAVE RESULTS AND MODELS
# ============================================================

# Save scalers
import pickle

with open('scalers.pkl', 'wb') as f:
    pickle.dump({'u': scaler_u, 'y': scaler_y}, f)

# Save model
torch.save(model.state_dict(), f'{MODEL_TYPE}_model.pth')

# Save system identification matrices
np.savez('system_matrices.npz', A=A_id, B=B_id, C=C_id, D=D_id)

print(f"\n✓ Results saved:")
print(f"  - {MODEL_TYPE} model: {MODEL_TYPE}_model.pth")
print(f"  - System matrices: system_matrices.npz")
print(f"  - Scalers: scalers.pkl")

## Section 9: Summary and Next Steps

In [ ]:
# ============================================================
# SUMMARY REPORT
# ============================================================

summary = f"""
╔══════════════════════════════════════════════════════════════════════════╗
║          DATA-DRIVEN DRONE ALTITUDE CONTROL - SUMMARY REPORT             ║
╚══════════════════════════════════════════════════════════════════════════╝

📊 DATA PROCESSING:
   • Input samples: {u_raw.shape[0]:,}
   • Output samples: {y_raw.shape[0]:,}
   • Sampling rate: 20 Hz (Ts = 50 ms)
   • Duration: {u_raw.shape[0] * 0.05:.1f} seconds

🤖 MACHINE LEARNING MODEL:
   • Architecture: {MODEL_TYPE}
   • Sequence length: {SEQ_LENGTH} steps
   • Training samples: {X_train.shape[0]:,}
   • Test samples: {X_test.shape[0]:,}
   • Final test loss: {test_losses[-1]:.6f}

🎯 SYSTEM IDENTIFICATION:
   • Method: Least-squares system identification
   • State dimension: 3 (position, velocity, acceleration)
   • Identified dynamics:
     - Acceleration decay: {A_id[2, 2]:.4f}
     - Control gain: {B_id[2, 0]:.4f}

🛰️  MPC CONTROLLER:
   • Horizon: 10 steps
   • Control law: u = -K χ̂ (LQI with augmented state)
   • Reference tracking: Multi-step altitude commands

📈 CONTROL PERFORMANCE:
   • Integral Absolute Error: {IAE:.4f} m·s
   • Maximum Error: {max_error:.4f} m
   • Settling Time (5%): {settling_time:.2f} s
   • Control Effort: {control_effort:.2f}

✅ OUTPUTS GENERATED:
   1. 01_data_exploration.png - Data statistics and phase plane
   2. 02_training_history.png - Neural network training curves
   3. 03_mpc_control_results.png - MPC tracking performance
   4. {MODEL_TYPE}_model.pth - Trained model weights
   5. system_matrices.npz - Identified A, B, C, D matrices
   6. scalers.pkl - Data normalization parameters

💡 RECOMMENDATIONS:
   • Adjust Q/R gains in LQI if tracking performance needs improvement
   • Try different model architectures (LSTM, GRU, Transformer) by changing MODEL_TYPE
   • Increase prediction horizon for smoother commands
   • Add disturbance rejection capabilities (e.g., wind) with observer
   • Implement adaptive control if system parameters drift over time
"""

print(summary)